In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']

## Interactivity in visualizations
This module should be useful for those of you wishing to incorporate interactivity into Jupyter notebook visualizations. We will go through some examples in Bokeh but the same ideas apply to matplotlib.

## When to use Jupyter notebooks
In contrast to techniques using D3, which tend to be focused on polished, *explanatory* visualizations for the end-user, Jupyter notebooks tend to be more useful for *exploration*. This is the time when running code modularly is useful because you're interested in the effect of changing a few lines of code without rerunning the entire script. We already discussed the rationale behind including visualization as part of your data exploration, and interactivity is a powerful tool to accomplish that objective, whether it's just you, or your team huddled around your computer.

Let's start with the same old time series data.

In [ ]:
import pandas as pd
import numpy as np

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.palettes import Spectral6
from IPython.html.widgets import *

In [ ]:
import requests
import simplejson as json

with open('small_data/goog.json') as raw_f:
    raw_data = raw_f.read()
    json_data = json.loads(raw_data)
    
df = pd.DataFrame(json_data['data'], columns=json_data['column_names'])
df.set_index(pd.DatetimeIndex(df['Date']), inplace=True)

In [ ]:
output_notebook()
TOOLS = "pan, box_zoom, wheel_zoom, reset, save"

## Layering information
While it is possible to include additional axes of data using color, thickness, texture, etc., it can be beneficial to use interactivity for this purpose.
* Gives the end user control, gets them engaged with the data.
* Allows for the base graph to be less cluttered and send a clearer message.
* "Feels" impressive. Interactive graphs make you feel like there's a vast amount of data which you're tapping into.

Most web-deployed interactive plots (or dashboards, as people like to call them) run on D3. You give these tools a large data source server-side, and the JavaScript can rapidly render the desired slice of the data. For polished tools, look at Bokeh server, Pyxley, or highcharts.

While it is possible to request new data as a function of a choice made by the end user, the bandwidth required to do this scales with how responsive you want the display to be and how detailed the data is. This is currently an area of active technology development.

A basic technique for adding interactivity to IPython notebooks is HTML widgets.

In [ ]:
def update(start=0, end=len(df), col='Close'):
    remove_from_tail = start
    remove_from_head = len(df) - end
    days = len(df)
    plot = figure(tools=TOOLS, title="Title", x_axis_label="date", x_axis_type="datetime")
    plot.line(df[col].index[days-end:days-start], df[col][days-end:days-start])
    show(plot)

In [ ]:
interact(update, start=(0,len(df)-1), end=(1,len(df)), col=('Open', 'High', 'Low', 'Close', 'Volume'))

*Exercise*: Change the update function so that it always plots a month of data, and the slider moves the window back and forth in time.

Now that you know how to do the basics, let's get a little fancier and overlay two stocks of the user's choice.

In [ ]:
def get_url(ticker):
    return 'https://www.quandl.com/api/v1/datasets/WIKI/%s.json' % ticker.upper()
    
def get_data(ticker):
    session = requests.Session()
    session.mount('http://', requests.adapters.HTTPAdapter(max_retries=2))
    raw_data = session.get(get_url(ticker)).content
    json_data = json.loads(raw_data)
    return pd.DataFrame(json_data['data'], columns=json_data['column_names'])

def update2(col='Volume', lag=20, ticker1='INTC', ticker2='AAPL'):
    tickers = [ticker1, ticker2]

    colorwheel = Spectral6
    color_idx = 0
    plot = figure(tools=TOOLS, title="Better Title", x_axis_label="date", x_axis_type="datetime")
    
    for ticker in tickers:
        if ticker not in dfs:
            dfs[ticker] = get_data(ticker)
            dfs[ticker].set_index(pd.DatetimeIndex(dfs[ticker]['Date']), inplace=True)
        df = dfs[ticker][:lag]
        plot.line(df[col].index, df[col], line_color=colorwheel[color_idx], line_width=2, legend=ticker)
        color_idx += 1
        if color_idx > len(colorwheel):
            color_idx = 0
    
    show(plot)

dfs = {}

In [ ]:
interact(update2, col=('Open', 'High', 'Low', 'Close', 'Volume'), lag=(5,360,5), ticker1='intc', ticker2='aapl')

### Next stop, Google Finance!

## Being creative
Don't be limited here - interactivity can affect not just data slicing but also the underlying source data, plot type, and more. You might even be able to cater to people viewing your chart on different devices.

For more ideas, styling, functionality, and examples, see [the documentation](http://nbviewer.ipython.org/github/quantopian/ipython/blob/master/examples/Interactive%20Widgets/Index.ipynb).

## Answering the second and third question
One of the great strengths of interactivity is the ability to conduct an entire discussion with your audience instead of just presenting them a static view of what you think they'll be interested in. The time sliders in the above example are nice, but most of the functionality comes from being able to select different columns (data at different scales) in the same time window and compare.

Exercise spoilers...

In [ ]:
def update(start=0, col='Close'):
    days = len(df)
    plot = figure(tools=TOOLS, title="Title", x_axis_label="date", x_axis_type="datetime")
    plot.line(df[col].index[days-start-20:days-start], df[col][days-start-20:days-start])
    show(plot)

In [ ]:
interact(update, start=(0,len(df)-20), col=('Open', 'High', 'Low', 'Close', 'Volume'))

*Copyright &copy; 2015 The Data Incubator.  All rights reserved.*